In [2]:
!pip install -U bitsandbytes


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import transformers
import inseq
import torch
import pandas as pd

/home/ada/humor/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
transcripts = pd.read_csv('/home/ada/humor/data/stand_up_dataset/standup_transcripts.csv')

In [5]:
transcripts.head()

,comedian,transcript
0,Anthony_Jeselnik,"When I was a kid, I used to fantasize about ge..."
1,Anthony_Jeselnik_2,No one should ever ask me to speak at anyone’...
2,Ali_Wong,"The last time I was at home in San Francisco, ..."
3,Ali_Wong_2,I need to have children to keep me company whe...
4,Chelsea_Peretti,A friend of a friend just posted like 500 enga...


In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ada/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
import nltk.data

def split_transcript(transcript):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = tokenizer.tokenize(transcript.strip())
    total_sentences = len(sentences)
    sentences_per_part = total_sentences // 4

    part1 = sentences[:sentences_per_part]
    part2 = sentences[sentences_per_part:2*sentences_per_part]
    part3 = sentences[2*sentences_per_part:3*sentences_per_part]
    part4 = sentences[3*sentences_per_part:]

    part1_text = ' '.join(part1)
    part2_text = ' '.join(part2)
    part3_text = ' '.join(part3)
    part4_text = ' '.join(part4)

    return [part1_text, part2_text, part3_text, part4_text]


In [8]:
transcript_ali_wong = transcripts.loc[transcripts['comedian'] == 'Ali_Wong', 'transcript'].values[0]
parts = split_transcript(transcript_ali_wong)

In [9]:
Q8BIT = transformers.BitsAndBytesConfig(load_in_8bit=True)

model = inseq.load_model(
    model="google/gemma-2b-it", 
    attribution_method="input_x_gradient",
    model_kwargs=dict(
        device_map="cuda:0",
        torch_dtype=torch.float32,
        trust_remote_code=False,
        low_cpu_mem_usage=True,
        attn_implementation="eager",
        quantization_config=Q8BIT,
    )
)

print("memory cost: ", model.model.get_memory_footprint() >> 20, "Mb")

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.26s/it]
The model is loaded in 8bit mode. The device cannot be changed after loading the model.


memory cost:  3890 Mb


In [10]:
# prompt = f"This is a stand-up comedy transcript: '''{parts[1]}'''\nExtract the key humorous lines and punchlines:"

# out = model.attribute(
# prompt,
# n_steps=16,
# batch_size=1,
# generation_args=dict(
#     max_new_tokens=65,
# )
# )
# ()
# out.show()

In [11]:
# out.sequence_attributions[0].target_attributions.sum(-1)

In [12]:
# for part in parts:
#     prompt = f"This is a stand-up comedy transcript: '''{part}'''\nExtract the key humorous lines and punchlines:"

#     out = model.attribute(
#     prompt,
#     n_steps=16,
#     batch_size=1,
#     device=model.device,
#     generation_args=dict(
#         max_new_tokens=65,
#     )
#     )

#     out.show()

In [13]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [14]:
!export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128

In [19]:
results = {}
for i in range(len(transcripts['comedian'])): 
    comedian = transcripts['comedian'][i]
    trans = transcripts['transcript'][i]
    
    parts = split_transcript(trans)
    results[comedian] = []
    
    for part in parts:
        prompt = f"This is a stand-up comedy transcript: '''{part}'''\nExtract the key humorous lines and punchlines:"

        out = model.attribute(
        prompt,
        n_steps=16,
        batch_size=1,
        device=model.device,
        generation_args=dict(
            max_new_tokens=65,
        )
        )
        results[comedian].append(out)


The model is loaded in 8bit mode. The device cannot be changed after loading the model.


Unused arguments during attribution: {'n_steps': 16}
Attributing with input_x_gradient...: 100%|██████████| 213/213 [00:26<00:00,  2.46it/s]
The model is loaded in 8bit mode. The device cannot be changed after loading the model.
The model is loaded in 8bit mode. The device cannot be changed after loading the model.
Unused arguments during attribution: {'n_steps': 16}
Attributing with input_x_gradient...: 100%|██████████| 152/152 [00:23<00:00,  2.80it/s]
The model is loaded in 8bit mode. The device cannot be changed after loading the model.
The model is loaded in 8bit mode. The device cannot be changed after loading the model.
Unused arguments during attribution: {'n_steps': 16}
Attributing with input_x_gradient...: 100%|██████████| 168/168 [00:23<00:00,  2.74it/s]
The model is loaded in 8bit mode. The device cannot be changed after loading the model.
The model is loaded in 8bit mode. The device cannot be changed after loading the model.
Unused arguments during attribution: {'n_steps': 

In [20]:
results

{'Anthony_Jeselnik': [FeatureAttributionOutput({
      sequence_attributions: list with 1 elements of type GradientFeatureAttributionSequenceOutput:[
          GradientFeatureAttributionSequenceOutput({
          source: list with 148 elements of type TokenWithId,
          target: list with 213 elements of type TokenWithId,
          source_attributions: None,
          target_attributions: torch.float32 tensor of shape [213, 65, 2048] on cpu,
          step_scores: {},
          sequence_scores: None,
          attr_pos_start: 148,
          attr_pos_end: 213,
      }
      ],
      step_attributions: None,
      info: {
          attribution_method: input_x_gradient,
          attr_pos_start: 148,
          attr_pos_end: 213,
          output_step_attributions: False,
          attribute_target: False,
          step_scores: list with 0 elements,
          exec_time: 26.424701,
          model_name: google/gemma-2b-it,
          model_class: GemmaForCausalLM,
          tokenizer_nam

In [18]:
results["Anthony_Jeselnik"][1].sequence_attributions[0].target_attributions.sum(-1)

tensor([[ 6.3210e-03, -2.7548e-02, -1.7815e-02,  ...,  3.0987e-03,
         -3.0957e-08, -2.0683e-04],
        [-7.8455e-02,  3.5944e-02, -3.9946e-02,  ...,  3.3455e-05,
         -2.8308e-07, -6.4833e-04],
        [ 2.0934e-03,  4.0179e-03, -7.0349e-03,  ..., -1.0699e-04,
          1.5696e-09,  4.7038e-05],
        ...,
        [        nan,         nan,         nan,  ...,         nan,
          3.5755e-07, -3.2188e-04],
        [        nan,         nan,         nan,  ...,         nan,
                 nan,  2.9316e-03],
        [        nan,         nan,         nan,  ...,         nan,
                 nan,         nan]])

In [25]:
file_path = "/home/ada/humor/data/input_gradient.json"
torch.save(results, file_path)
